In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.stats import ttest_rel, sem
from sklearn.metrics import confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb, ListedColormap
from matplotlib.patches import Circle

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
file_name = repo_path / 'data/covert-search/large-field/p4_data_AZ_t1.mat'
data = loadmat(file_name)
print(data.keys())

In [ ]:
taget_amplitude = data['targetAmplitude']
target_location = data['tLocation']
spot_center = data['spotCenters']
human_response = data['hResponse']
n_location = data['nLocations'][0][0]
spot_diameter = data['spotLength'][0][0]
stimulus_design_size= data['totalLength'][0][0]
monitor_width = data['monitorPx'][0][0]
monitor_height = data['monitorPx'][0][1]
ppd = data['ppd'][0][0]

In [ ]:
shifted_spot_center = spot_center.copy()
shifted_spot_center[:,0] += (monitor_height - stimulus_design_size)//2
shifted_spot_center[:,1] += (monitor_width - stimulus_design_size)//2

accurate_response = target_location == human_response

In [ ]:
list_spot_region = [np.zeros(monitor_width,monitor_height)] * (n_location-1)
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

for index_location in range(n_location-1):
     list_spot_region[index_location] = (pixel_row - shifted_spot_center[index_location, 0])**2+(pixel_col - shifted_spot_center[index_location, 1])**2<= spot_diameter**2/4

In [ ]:
stimulus_region = np.zeros((monitor_height, monitor_width))

for index_location in range(n_location-1):
    stimulus_region[list_spot_region[index_location]] = 1

In [ ]:
fig, ax = plt.subplots()
ax.imshow(stimulus_region)

for index_location in range(n_location-1):
    ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center')

circle = Circle((monitor_width//2, monitor_height//2), spot_diameter*1.5, edgecolor='r', facecolor='none', linewidth=3)
ax.add_patch(circle)

In [ ]:
array_eccentral_distance = np.zeros((n_location-1,))

for index_location in range(n_location-1):
    array_eccentral_distance[index_location] = np.sqrt((spot_center[index_location, 0] - stimulus_design_size//2)**2+(spot_center[index_location, 1] - stimulus_design_size//2)**2)

array_eccentral_distance /= ppd

print(array_eccentral_distance)

In [ ]:
pixel_precision_array_eccentral_distance = array_eccentral_distance.copy()

for index_d1, distance1 in enumerate(np.unique(array_eccentral_distance)):
    for index_d2, distance2 in enumerate(np.unique(array_eccentral_distance)[index_d1+1:]):
        if distance1 != distance2 and np.abs(distance2 - distance1) < 0.5:
            print(index_d1, distance1, index_d2, distance2)
            pixel_precision_array_eccentral_distance[array_eccentral_distance==distance2] = distance1

print(pixel_precision_array_eccentral_distance)

In [ ]:
eccentral_distance_sorted_indexes = np.argsort(pixel_precision_array_eccentral_distance, kind='stable')
extra_eccentral_distance_sorted_indexes = np.insert(eccentral_distance_sorted_indexes+1,0,0)
print(extra_eccentral_distance_sorted_indexes)

In [ ]:
accurate_response = target_location == human_response
confusion_mat = confusion_matrix(human_response.flatten(), target_location.flatten(), labels=range(n_location))[extra_eccentral_distance_sorted_indexes, :][:, extra_eccentral_distance_sorted_indexes]
array_index_location = np.arange(n_location)[extra_eccentral_distance_sorted_indexes]

In [ ]:
correct_rejection = confusion_mat[0][0]
second_largest_confusion_mat = np.max(np.delete(confusion_mat, np.where(confusion_mat == correct_rejection)))
plotted_confusion_mat = confusion_mat.copy()
plotted_confusion_mat[0][0] = second_largest_confusion_mat
print(second_largest_confusion_mat)

In [ ]:
indexes_near = np.arange(1,n_location)[pixel_precision_array_eccentral_distance < spot_diameter*1.5/ppd]
indexes_far = np.arange(1,n_location)[pixel_precision_array_eccentral_distance > spot_diameter*1.5/ppd]
print(indexes_near, indexes_far)

In [ ]:
near_stimulus_region = np.zeros((monitor_height, monitor_width))

for index_near in indexes_near:
    near_stimulus_region[list_spot_region[index_near-1]] = 1

In [ ]:
fig, ax = plt.subplots()
ax.imshow(near_stimulus_region)

for index_near in indexes_near:
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{index_near}', ha='center', va='center')

In [ ]:
accuracy_near = accurate_response[np.isin(target_location, indexes_near)].mean()
print(accuracy_near)

In [ ]:
fig, ax = plt.subplots(figsize=(15,12))
im = ax.imshow(plotted_confusion_mat)

for index_target_location in array_index_location:
    for index_human_response in array_index_location:
        if not (-index_target_location == 0 and index_human_response == 0):
            ax.text(index_human_response, index_target_location, f'{confusion_mat[index_target_location, index_human_response]:.0f}', ha='center', va='center')

ax.annotate(f'{correct_rejection:.0f}', xy=(0, 0), xytext=(-0.5,-1), arrowprops=dict(facecolor='black', shrink=0.05))

ax.set(xticks=np.arange(n_location), yticks=np.arange(n_location), xticklabels=array_index_location, yticklabels=array_index_location, xlabel='Target location sorted by eccentral distance', ylabel='Human response sorted by eccentral distance')

inner_ax = fig.add_axes([0.23, 0.3, 0.2, 0.32])
inner_ax.imshow(stimulus_region)
inner_ax.set(xticks=[],yticks=[])

for index_location in range(n_location-1):
    inner_ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center', fontsize=16)

plt.show()

In [ ]:
local_hit_region = np.empty((monitor_height, monitor_width))

local_hit = np.array([((target_location == index_near) & (human_response == index_near)).sum() / (target_location == index_near).sum() for index_near in indexes_near])

for index_index_near, index_near in enumerate(indexes_near):
    local_hit_region[list_spot_region[index_near-1]] = local_hit[index_index_near]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_hit_region)

for index_index_near, index_near in enumerate(indexes_near):
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{local_hit[index_index_near]:.3f}', ha='center', va='center')

In [ ]:
local_miss_region = np.empty((monitor_height, monitor_width))

local_miss = np.array([((target_location == index_near) & (human_response == 0)).sum() / (target_location == index_near).sum()  for index_near in indexes_near])

for index_index_near, index_near in enumerate(indexes_near):
    local_miss_region[list_spot_region[index_near-1]] = local_miss[index_index_near]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_miss_region)

for index_index_near, index_near in enumerate(indexes_near):
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{local_miss[index_index_near]:.3f}', ha='center', va='center')

In [ ]:
local_fa_region = np.empty((monitor_height, monitor_width))

local_fa = np.array([((target_location == 0) & (human_response == index_near)).sum() / (target_location == 0).sum()  for index_near in indexes_near])

for index_index_near, index_near in enumerate(indexes_near):
    local_fa_region[list_spot_region[index_near-1]] = local_fa[index_index_near]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_fa_region)

for index_index_near, index_near in enumerate(indexes_near):
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{local_fa[index_index_near]:.3f}', ha='center', va='center')

In [ ]:
cr = ((target_location == 0) & (human_response == 0)).sum() / (target_location == 0).sum()

print(cr)

In [ ]:
local_hit_region = np.empty((monitor_height, monitor_width))

local_hit_count = np.array([((target_location == index_near) & (human_response == index_near)).sum() for index_near in indexes_near])

for index_index_near, index_near in enumerate(indexes_near):
    local_hit_region[list_spot_region[index_near-1]] = local_hit_count[index_index_near]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_hit_region)

for index_index_near, index_near in enumerate(indexes_near):
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{local_hit_count[index_index_near]:.0f}', ha='center', va='center')

In [ ]:
local_miss_region = np.empty((monitor_height, monitor_width))

local_miss_count = np.array([((target_location == index_near) & (human_response == 0)).sum() for index_near in indexes_near])

for index_index_near, index_near in enumerate(indexes_near):
    local_miss_region[list_spot_region[index_near-1]] = local_miss_count[index_index_near]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_miss_region)

for index_index_near, index_near in enumerate(indexes_near):
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{local_miss_count[index_index_near]:.0f}', ha='center', va='center')

In [ ]:
local_fa_region = np.empty((monitor_height, monitor_width))

local_fa_count = np.array([((target_location == 0) & (human_response == index_near)).sum() for index_near in indexes_near])

for index_index_near, index_near in enumerate(indexes_near):
    local_fa_region[list_spot_region[index_near-1]] = local_fa_count[index_index_near]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_fa_region)

for index_index_near, index_near in enumerate(indexes_near):
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{local_fa_count[index_index_near]:.0f}', ha='center', va='center')

In [ ]:
local_fh_from_region = np.empty((monitor_height, monitor_width))

local_fh_from_count = np.array([((target_location == index_near) & (human_response != index_near) & (human_response != 0)).sum() for index_near in indexes_near])

for index_index_near, index_near in enumerate(indexes_near):
    local_fh_from_region[list_spot_region[index_near-1]] = local_fh_from_count[index_index_near]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_fh_from_region)

for index_index_near, index_near in enumerate(indexes_near):
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{local_fh_from_count[index_index_near]:.0f}', ha='center', va='center')

In [ ]:
local_fh_to_region = np.empty((monitor_height, monitor_width))

local_fh_to_count = np.array([((target_location != 0) & (target_location != index_near) & (human_response == index_near)).sum() for index_near in indexes_near])

for index_index_near, index_near in enumerate(indexes_near):
    local_fh_to_region[list_spot_region[index_near-1]] = local_fh_to_count[index_index_near]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_fh_to_region)

for index_index_near, index_near in enumerate(indexes_near):
    ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{local_fh_to_count[index_index_near]:.0f}', ha='center', va='center')

In [ ]:
orientation_hsv_color_map = np.zeros((n_location-1,3))

for index_location in range(n_location-1):
    dy = spot_center[index_location, 0] - stimulus_design_size//2
    dx = stimulus_design_size//2 - spot_center[index_location, 1]
    orientation_hsv_color_map[index_location,0] = (1 - np.arctan2(dy,dx) / np.pi) / 2

    orientation_hsv_color_map[index_location,1] = 1

    if dx == 0 and dy == 0:
        orientation_hsv_color_map[index_location,2] = 0
    else:
        orientation_hsv_color_map[index_location,2] = 0.75

rgb_orientation_color_map = hsv_to_rgb(orientation_hsv_color_map)

In [ ]:
large_field_near_field_hit = np.array([0.73913043, 0.73684211, 0.4, 0.61111111, 0.34482759, 0.08333333, 0.05555556, 0.125, 0.66666667, 0.55555556, 0.75, 0.25, 0.85, 0.7 , 0.78571429, 0.39285714, 0.32, 0.21428571, 0.45454545])[indexes_near-1]

In [ ]:
fig, ax = plt.subplots()
before = large_field_near_field_hit
after = local_hit

for index_pair, point_before in enumerate(before):
    ax.plot([0, 1], [point_before, after[index_pair]], '-', linewidth=3, color=rgb_orientation_color_map[indexes_near[index_pair]-1,:], alpha=0.6)

ax.errorbar([0, 1], [before.mean(), after.mean()], yerr=[sem(before), sem(after)], color='black', linewidth=7)

ax.set(ylabel='Hit', xticks=[0, 1])
ax.set_xticklabels(['16-degree field', '8-degree field'], fontsize=24)

inner_ax = fig.add_axes([0.15, 0.55, 0.24, 0.384])
inner_ax.imshow(near_stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_near in indexes_near:
    circle = Circle((shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0]), spot_diameter/2, color=rgb_orientation_color_map[index_near-1], linewidth=1, alpha=0.6)
    inner_ax.add_patch(circle)
    
    inner_ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{index_near}', ha='center', va='center', c='w', fontsize=16)

t_val, p_val = ttest_rel(after, before)
print(f'Mean difference: {(after-before).mean():.3f}; (paired sample t-test) t-statistic: {t_val:.3f}, p-value: {p_val:.5f}.')

In [ ]:
large_field_near_field_miss = np.array([0.2173913 , 0.26315789, 0.56, 0.27777778, 0.4137931, 0.70833333, 0.83333333, 0.875, 0.26666667, 0.44444444, 0.25, 0.54166667, 0.15, 0.3 , 0.17857143, 0.53571429, 0.6 , 0.64285714, 0.5 ])[indexes_near-1]

In [ ]:
fig, ax = plt.subplots()
before = large_field_near_field_miss
after = local_miss

for index_pair, point_before in enumerate(before):
    ax.plot([0, 1], [point_before, after[index_pair]], '-', linewidth=3, color=rgb_orientation_color_map[indexes_near[index_pair]-1,:], alpha=0.6)

ax.errorbar([0, 1], [before.mean(), after.mean()], yerr=[sem(before), sem(after)], color='black', linewidth=7)

ax.set(ylabel='Miss', xticks=[0, 1])
ax.set_xticklabels(['16-degree field', '8-degree field'], fontsize=24)

inner_ax = fig.add_axes([0.15, 0.05, 0.24, 0.384])
inner_ax.imshow(near_stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_near in indexes_near:
    circle = Circle((shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0]), spot_diameter/2, color=rgb_orientation_color_map[index_near-1], linewidth=1, alpha=0.6)
    inner_ax.add_patch(circle)
    
    inner_ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{index_near}', ha='center', va='center', c='w', fontsize=16)

t_val, p_val = ttest_rel(after, before)
print(f'Mean difference: {(after-before).mean():.3f}; (paired sample t-test) t-statistic: {t_val:.3f}, p-value: {p_val:.5f}.')

In [ ]:
large_field_near_field_fa = np.array([0.01507538, 0.00502513, 0.00251256, 0.01005025, 0.00251256,0.00502513, 0.00251256, 0.00251256, 0.01256281, 0.01256281,0.01507538, 0.00502513, 0.00753769, 0.01256281, 0.00753769,0.00251256, 0.01758794, 0.01005025, 0.01005025])[indexes_near-1]

In [ ]:
fig, ax = plt.subplots()
before = large_field_near_field_fa
after = local_fa

for index_pair, point_before in enumerate(before):
    ax.plot([0, 1], [point_before, after[index_pair]], '-', linewidth=3, color=rgb_orientation_color_map[indexes_near[index_pair]-1,:], alpha=0.6)

ax.errorbar([0, 1], [before.mean(), after.mean()], yerr=[sem(before), sem(after)], color='black', linewidth=7)

ax.set(ylabel='False alarm', xticks=[0, 1])
ax.set_xticklabels(['16-degree field', '8-degree field'], fontsize=24)

inner_ax = fig.add_axes([0.15, 0.55, 0.24, 0.384])
inner_ax.imshow(near_stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_near in indexes_near:
    circle = Circle((shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0]), spot_diameter/2, color=rgb_orientation_color_map[index_near-1], linewidth=1, alpha=0.6)
    inner_ax.add_patch(circle)
    
    inner_ax.text(shifted_spot_center[index_near-1, 1], shifted_spot_center[index_near-1, 0], f'{index_near}', ha='center', va='center', c='w', fontsize=16)

t_val, p_val = ttest_rel(after, before)
print(f'Mean difference: {(after-before).mean():.3f}; (paired sample t-test) t-statistic: {t_val:.3f}, p-value: {p_val:.5f}.')